In [8]:
# Importación de Librerías Esenciales

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


<div class="alert alert-block alert-info">
    <h1>Taller Práctico: Regresión Lineal, Ridge y Lasso</h1>
    <p><strong>Objetivos de la sesión:</strong></p>
    <ul>
        <li>Aplicar los conceptos de regresión lineal en un problema práctico.</li>
        <li>Implementar y comparar modelos de Regresión Lineal, Ridge y Lasso.</li>
        <li>Utilizar la validación cruzada para encontrar el hiperparámetro de regularización óptimo.</li>
        <li>Evaluar el rendimiento de los modelos utilizando métricas como MSE, MAE y R².</li>
    </ul>
    <p><strong>Dataset:</strong> Utilizaremos el dataset "Ames Housing", que contiene información sobre la venta de casas en Ames, Iowa. Nuestro objetivo será predecir el precio de venta (<code>SalePrice</code>) de las casas.</p>
</div>

### 1. Carga y Exploración de Datos (EDA)

<p>El primer paso en cualquier proyecto de Machine Learning es entender nuestros datos. Cargaremos el dataset, veremos su estructura y realizaremos algunas visualizaciones iniciales.</p>

In [2]:
# Cargar el dataset Ames Housing desde OpenML
housing = fetch_openml(name="house_prices", as_frame=True)
df = housing.frame

# Seleccionamos un subconjunto de características numéricas para simplificar el taller
# y la variable objetivo 'SalePrice'
numeric_features = ['GrLivArea', 'OverallQual', 'YearBuilt', 'TotalBsmtSF', 'FullBath', 'GarageCars']
target_variable = 'SalePrice'

# Creamos un DataFrame más pequeño y manejamos valores faltantes de forma simple
df_subset = df[numeric_features + [target_variable]].dropna()

print(f"\nDimensiones del subconjunto de datos: {df_subset.shape}")
print("\nDescripción estadística del subconjunto:")
print(df_subset.describe())


Dimensiones del subconjunto de datos: (1460, 7)

Descripción estadística del subconjunto:
         GrLivArea  OverallQual    YearBuilt  TotalBsmtSF     FullBath  \
count  1460.000000  1460.000000  1460.000000  1460.000000  1460.000000   
mean   1515.463699     6.099315  1971.267808  1057.429452     1.565068   
std     525.480383     1.382997    30.202904   438.705324     0.550916   
min     334.000000     1.000000  1872.000000     0.000000     0.000000   
25%    1129.500000     5.000000  1954.000000   795.750000     1.000000   
50%    1464.000000     6.000000  1973.000000   991.500000     2.000000   
75%    1776.750000     7.000000  2000.000000  1298.250000     2.000000   
max    5642.000000    10.000000  2010.000000  6110.000000     3.000000   

        GarageCars      SalePrice  
count  1460.000000    1460.000000  
mean      1.767123  180921.195890  
std       0.747315   79442.502883  
min       0.000000   34900.000000  
25%       1.000000  129975.000000  
50%       2.000000  163000

<p>Ahora, visualicemos la distribución de nuestra variable objetivo y la relación entre el área de la vivienda y su precio. La distribución de precios está sesgada, por lo que una transformación logarítmica será útil.</p>

In [3]:
# Ahora puedes mostrar los gráficos sin error
fig_hist = px.histogram(df_subset, x='SalePrice', nbins=50, title='Distribución del Precio de Venta de las Casas (Original)')
fig_hist.update_layout(xaxis_title='Precio de Venta (USD)', yaxis_title='Frecuencia')
fig_hist.show()

fig_hist_log = px.histogram(x=np.log1p(df_subset['SalePrice']), nbins=50, title='Distribución del log(Precio de Venta)')
fig_hist_log.update_layout(xaxis_title='log(1 + Precio de Venta)', yaxis_title='Frecuencia')
fig_hist_log.show()

### 2. Preparación de Datos

<p>Ahora aplicaremos los cambios necesarios: división de datos, transformación logarítmica del objetivo y escalado de características.</p>

In [9]:
# 1. Definir características (X) y objetivo (y)
X = df_subset[numeric_features]
y_raw = df_subset[target_variable] # Guardamos el 'y' original para evaluación final

# !! CAMBIO CLAVE !!
# Aplicamos la transformación logarítmica a la variable objetivo 'y'
# Esto estabiliza la varianza y normaliza la escala del problema.
y = np.log1p(y_raw)

# 2. Dividir en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del set de entrenamiento: {X_train.shape}")
print(f"Tamaño del set de prueba: {X_test.shape}")

# 3. Escalar las características numéricas
scaler = StandardScaler()

# Ajustamos el escalador SOLO con los datos de entrenamiento para evitar fuga de datos (data leakage)
X_train_scaled = scaler.fit_transform(X_train)
# Aplicamos la misma transformación a los datos de prueba
X_test_scaled = scaler.transform(X_test)

# Convertimos los arrays de numpy de vuelta a DataFrames de pandas para mejor legibilidad
X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_features)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=numeric_features)

Tamaño del set de entrenamiento: (1168, 6)
Tamaño del set de prueba: (292, 6)


### 3. Regresión Lineal (Mínimos Cuadrados Ordinarios)

<p>Entrenamos nuestro modelo base con los datos transformados. El MSE será un número pequeño y manejable.</p>

In [5]:
# Crear y entrenar el modelo de Regresión Lineal
ols_model = LinearRegression()
ols_model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba (las predicciones estarán en escala logarítmica)
y_pred_ols = ols_model.predict(X_test_scaled)

# Evaluar el modelo
mse_ols = mean_squared_error(y_test, y_pred_ols)
r2_ols = r2_score(y_test, y_pred_ols)

print("--- Evaluación del Modelo de Regresión Lineal (OLS) ---")
print(f"Error Cuadrático Medio (MSE en escala log): {mse_ols:.4f}")
print(f"Coeficiente de Determinación (R²): {r2_ols:.4f}")

# Ver los coeficientes del modelo
coef_df = pd.DataFrame(ols_model.coef_, index=numeric_features, columns=['Coeficiente_OLS'])
print("\nCoeficientes del modelo OLS:")
print(coef_df)

--- Evaluación del Modelo de Regresión Lineal (OLS) ---
Error Cuadrático Medio (MSE en escala log): 0.0303
Coeficiente de Determinación (R²): 0.8375

Coeficientes del modelo OLS:
             Coeficiente_OLS
GrLivArea           0.126585
OverallQual         0.138404
YearBuilt           0.074407
TotalBsmtSF         0.044540
FullBath           -0.006889
GarageCars          0.071504


### 4. Regresión con Regularización: Ridge (L2) y Lasso (L1)

<p>Con el objetivo ya transformado, ahora `alpha` tendrá el efecto esperado. Usaremos un valor pequeño para ver el contraste.</p>

In [10]:
# --- Ridge Regression (L2) ---
ridge_model = Ridge(alpha=100.) # Alpha grande
ridge_model.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(X_test_scaled)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("--- Evaluación del Modelo Ridge (alpha=100.) ---")
print(f"Error Cuadrático Medio (MSE en escala log): {mse_ridge:.4f}")
print(f"Coeficiente de Determinación (R²): {r2_ridge:.4f}")

# --- Lasso Regression (L1) ---
lasso_model = Lasso(alpha=0.1) # Alpha pequeño
lasso_model.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_model.predict(X_test_scaled)

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("\n--- Evaluación del Modelo Lasso (alpha=0.1) ---")
print(f"Error Cuadrático Medio (MSE en escala log): {mse_lasso:.4f}")
print(f"Coeficiente de Determinación (R²): {r2_lasso:.4f}")

# Comparar coeficientes
coef_df['Coef_Ridge_alpha100'] = ridge_model.coef_
coef_df['Coef_Lasso_alpha0.1'] = lasso_model.coef_
print("\n--- Comparación de Coeficientes ---")
print(coef_df.round(4))

--- Evaluación del Modelo Ridge (alpha=100.) ---
Error Cuadrático Medio (MSE en escala log): 0.0310
Coeficiente de Determinación (R²): 0.8340

--- Evaluación del Modelo Lasso (alpha=0.1) ---
Error Cuadrático Medio (MSE en escala log): 0.0579
Coeficiente de Determinación (R²): 0.6895

--- Comparación de Coeficientes ---
             Coeficiente_OLS  Coef_Ridge_alpha100  Coef_Lasso_alpha0.1
GrLivArea             0.1266               0.1120               0.0567
OverallQual           0.1384               0.1285               0.1479
YearBuilt             0.0744               0.0673               0.0050
TotalBsmtSF           0.0445               0.0499               0.0060
FullBath             -0.0069               0.0080               0.0000
GarageCars            0.0715               0.0723               0.0486


<p>¡Ahora sí! Observa cómo los coeficientes de Ridge son sistemáticamente más pequeños (encogidos) que los de OLS. Y más importante, mira los coeficientes de Lasso: con un `alpha` de solo 0.1, ¡ya ha reducido el coeficiente de `FullBath` casi a cero, demostrando su poder de selección de características!</p>

### 5. Selección de Hiperparámetros con Validación Cruzada

<p>Finalmente, usamos `RidgeCV` y `LassoCV` para encontrar el `alpha` óptimo de manera automática.</p>

In [ ]:
from sklearn.linear_model import RidgeCV

# Definir un rango de alphas para probar
alphas = np.logspace(-4, 2, 100)

# --- RidgeCV ---
# ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv = RidgeCV(alphas=alphas, store_cv_results=True)
ridge_cv.fit(X_train_scaled, y_train)

print("--- Búsqueda del mejor Alpha para Ridge ---")
print(f"Mejor alpha encontrado para Ridge: {ridge_cv.alpha_:.4f}")

# Evaluar el modelo Ridge final con el mejor alpha
y_pred_ridge_cv = ridge_cv.predict(X_test_scaled)
mse_ridge_cv = mean_squared_error(y_test, y_pred_ridge_cv)
r2_ridge_cv = r2_score(y_test, y_pred_ridge_cv)
print(f"MSE de Ridge con CV: {mse_ridge_cv:.4f}")
print(f"R² de Ridge con CV: {r2_ridge_cv:.4f}")

# --- LassoCV ---
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_cv.fit(X_train_scaled, y_train)

print("\n--- Búsqueda del mejor Alpha para Lasso ---")
print(f"Mejor alpha encontrado para Lasso: {lasso_cv.alpha_:.4f}")

# Evaluar el modelo Lasso final
y_pred_lasso_cv = lasso_cv.predict(X_test_scaled)
mse_lasso_cv = mean_squared_error(y_test, y_pred_lasso_cv)
r2_lasso_cv = r2_score(y_test, y_pred_lasso_cv)
print(f"MSE de Lasso con CV: {mse_lasso_cv:.4f}")
print(f"R² de Lasso con CV: {r2_lasso_cv:.4f}")

# Añadir los coeficientes finales al DataFrame
coef_df['Coef_Ridge_CV'] = ridge_cv.coef_
coef_df['Coef_Lasso_CV'] = lasso_cv.coef_
print("\n--- Comparación Final de Coeficientes ---")
print(coef_df.round(4))

--- Búsqueda del mejor Alpha para Ridge ---
Mejor alpha encontrado para Ridge: 86.9749
MSE de Ridge con CV: 0.0309
R² de Ridge con CV: 0.8346

--- Búsqueda del mejor Alpha para Lasso ---
Mejor alpha encontrado para Lasso: 0.0043
MSE de Lasso con CV: 0.0307
R² de Lasso con CV: 0.8357

--- Comparación Final de Coeficientes ---
             Coeficiente_OLS  Coef_Ridge_alpha100  Coef_Lasso_alpha0.1  \
GrLivArea             0.1266               0.1120               0.0567   
OverallQual           0.1384               0.1285               0.1479   
YearBuilt             0.0744               0.0673               0.0050   
TotalBsmtSF           0.0445               0.0499               0.0060   
FullBath             -0.0069               0.0080               0.0000   
GarageCars            0.0715               0.0723               0.0486   

             Coef_Ridge_CV  Coef_Lasso_CV  
GrLivArea           0.1135         0.1199  
OverallQual         0.1297         0.1385  
YearBuilt           0.

### 6. Desafíos para Experimentar

<p>Ahora es tu turno de experimentar. Responde a las siguientes preguntas modificando el código anterior:</p>
<ol>
    <li><strong>Añadir más características:</strong> Elige otras 2 o 3 características numéricas del `df` original, añádelas a la lista `numeric_features` y vuelve a ejecutar todo el notebook. ¿Mejora el R² del modelo final?</li>
    <li><strong>Interpretación de coeficientes:</strong> Observa los coeficientes finales del modelo `LassoCV`. ¿Qué característica parece ser la más importante según este modelo? ¿Cuáles ha descartado (coeficiente cercano a cero)?</li>
    <li><strong>Evaluación en escala original:</strong> Las predicciones (`y_pred_lasso_cv`) están en escala logarítmica. Usa `np.expm1()` para convertirlas de nuevo a dólares. Luego, calcula el Error Absoluto Medio (MAE) entre las predicciones revertidas y el `y_test` original (que deberías haber guardado como `y_raw`). ¿Cuál es el error promedio en dólares de tu mejor modelo?</li>
</ol>

### 7. Ejercicios Propuestos

<div class="alert alert-block alert-warning">
<h4>Ejercicios para Solidificar Conceptos</h4>
<ol>
    <li><strong>Evaluación de Residuos:</strong> Para el mejor modelo que encontraste (probablemente RidgeCV o LassoCV), calcula los residuos ($y_{test} - \hat{y}$) y crea un gráfico de dispersión de los valores predichos vs. los residuos. ¿Observas algún patrón? Un patrón podría indicar que la relación no es puramente lineal.</li>
    <li><strong>Cambiar la Métrica de CV:</strong> En `LassoCV` y `RidgeCV`, el scoring por defecto es el error cuadrático negativo. Investiga cómo podrías usar el "mean_absolute_error" en su lugar. ¿Cambia el `alpha` seleccionado?</li>
    <li><strong>Implementar K-Fold a Mano:</strong> En lugar de usar `RidgeCV`, instancia un `KFold(n_splits=5)`, itera sobre los pliegues, entrena un modelo `Ridge` para un alpha fijo en cada pliegue y calcula el error promedio. Compara tu resultado con el de la clase `RidgeCV`.</li>
    <li><strong>Características Polinómicas:</strong> Sospechamos que la relación entre `GrLivArea` y `SalePrice` no es perfectamente lineal. Usa `sklearn.preprocessing.PolynomialFeatures` para crear una característica `GrLivArea` al cuadrado y añádela al modelo. ¿Mejora el R²?</li>
    <li><strong>Impacto de Outliers:</strong> Encuentra la casa más cara en el `df_subset`. Elimínala y vuelve a entrenar el modelo OLS (con el target logarítmico). ¿Cómo cambian los coeficientes y el R²?</li>
    <li><strong>Comparar MAE vs MSE:</strong> Calcula el `mean_absolute_error` (MAE) para todos los modelos finales (OLS, RidgeCV, LassoCV) en la escala logarítmica. ¿Por qué el MAE es menor que el MSE?</li>
    <li><strong>Importancia de Características:</strong> Ordena los coeficientes del modelo `RidgeCV` final de mayor a menor en valor absoluto. ¿Cuáles son las 3 características más influyentes según este modelo?</li>
    <li><strong>Efecto del Alpha en Lasso:</strong> Crea un gráfico que muestre cómo cambian los coeficientes de Lasso a medida que `alpha` aumenta. Puedes usar la información almacenada en `lasso_cv.path_`.</li>
    <li><strong>Regresión Lineal Simple vs. Múltiple:</strong> Entrena un modelo de regresión lineal simple para cada una de las características en `numeric_features` por separado. Compara el coeficiente de cada característica en su modelo simple con su coeficiente en el modelo múltiple (OLS). ¿Por qué son diferentes?</li>
    <li><strong>Modelo para Producción:</strong> Si tuvieras que elegir uno de los modelos entrenados para ponerlo en producción y predecir precios de casas para un cliente, ¿cuál elegirías y por qué? Justifica tu respuesta basándote en el rendimiento (R², MSE), la interpretabilidad (coeficientes) y la simplicidad del modelo.</li>
</ol>
</div>